This first part pre-processes EC data and saves u,v,w,q,C,P,Tsonic and hourly averaged data with the desired format.\
data is from date(2020,9,25) to date(2021,4,23)\
Note: GroupC data missing from date(2021,1,15) to date(2021,2,21), and date(2020,12,16)\
2021-8-24 created by Yue

In [1]:
# Turn off annoying warnings
import warnings
warnings.filterwarnings('ignore')
# Make sure the environment is good
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import os.path
import pickle
import json
import datetime
import time
import matplotlib.dates as dates
from datetime import date, timedelta

DATA_DIRECTORY = "/projectnb/urbanclimate/public/Idaho_2020/tsdata/" 
OUT_DIRECTORY = "/projectnb/urbanclimate/yueqin/idaho_ec_jupyter/processed_data/"

## Define function

In [2]:
def date_list(sdate,edate):
    """method used for creating date list"""
    delta = edate - sdate       # as timedelta
    day = [sdate+timedelta(days=x) for x in range(delta.days+1)]
    return day

In [4]:
sonum    =12                       # number of sonic
z        =[1.2,2,3.5,6,8.2,12.8,15.8,23,30.3,40.2,50.6,60.5];                 # height of sonic above ground, m
frequency=10                   # sampling frequency, Hz
time_avg =3600                  # average time, s
i   = 1     # loop for number of files
kd   = 1     # loop for saves in all files
rpat = time_avg*frequency           # number of lines for a loop
u_header=['Ux_C1','Ux_C2','Ux_C3','Ux_C4','Ux_B1','Ux_B2','Ux_B3','Ux_B4','Ux_A1','Ux_A2','Ux_A3','Ux_A4']
v_header=['Uy_C1','Uy_C2','Uy_C3','Uy_C4','Uy_B1','Uy_B2','Uy_B3','Uy_B4','Uy_A1','Uy_A2','Uy_A3','Uy_A4']
w_header=['Uz_C1','Uz_C2','Uz_C3','Uz_C4','Uz_B1','Uz_B2','Uz_B3','Uz_B4','Uz_A1','Uz_A2','Uz_A3','Uz_A4']
Tsonic_header=['Ts_C1','Ts_C2','Ts_C3','Ts_C4','Ts_B1','Ts_B2','Ts_B3','Ts_B4','Ts_A1','Ts_A2','Ts_A3','Ts_A4']
diag_header=['diag_sonic_C1','diag_sonic_C2','diag_sonic_C3','diag_sonic_C4','diag_sonic_B1','diag_sonic_B2','diag_sonic_B3','diag_sonic_B4',
            'diag_sonic_A1','diag_sonic_A2','diag_sonic_A3','diag_sonic_A4']
co2_header=['co2_C1','co2_C2','co2_C3','co2_C4','co2_B1','co2_B2','co2_B3','co2_B4','co2_A1','co2_A2','co2_A3','co2_A4']
h2o_header=['h2o_C1','h2o_C2','h2o_C3','h2o_C4','h2o_B1','h2o_B2','h2o_B3','h2o_B4','h2o_A1','h2o_A2','h2o_A3','h2o_A4']
sig_header=['sig_irga_C1','sig_irga_C2','sig_irga_C3','sig_irga_C4','sig_irga_B1','sig_irga_B2','sig_irga_B3',
            'sig_irga_B4','sig_irga_A1','sig_irga_A2','sig_irga_A3','sig_irga_A4']
press_header=['Press_irga_C1','Press_irga_C2','Press_irga_C3','Press_irga_C4','Press_irga_B1','Press_irga_B2','Press_irga_B3',
            'Press_irga_B4','Press_irga_A1','Press_irga_A2','Press_irga_A3','Press_irga_A4']
# print(len(month))

out_var=['u_ins','v_ins','w_ins','q_ins','C_ins','P_ins','Tsonic_ins',
         'diag_csat_ins','sig_irga_ins'] #,'date_num_ins']
# out_var=['diag_csat_ins','sig_irga_ins']

# set up saving controller
do_saving = 0

# compute the total days as selected in files

In [8]:
d_total = 0
# dys  = list(range(1,32))
Sdate = date(2021,1,15)
Edate = date(2021,2,21)
ds = date_list(Sdate,Edate)
for day in ds:
    strday = str(day.strftime("%Y%m%d"))       
    # read in the input data            
    fp_stats1 = DATA_DIRECTORY +'Group_C/ec_flux_' + strday + '_0000.dat'
    fp_stats2 = DATA_DIRECTORY +'Group_B/ec_flux_' + strday + '_0000.dat'
    fp_stats3 = DATA_DIRECTORY +'Group_A/ec_flux_' + strday + '_0000.dat'
    # if os.path.isfile(fp_stats2) & os.path.isfile(fp_stats3):
    if os.path.isfile(fp_stats1) & os.path.isfile(fp_stats2) & os.path.isfile(fp_stats2):
        d_total += 1
print('total day:'+str(d_total))

total day:0


# Reorganize data

In [ ]:
%%time
t_id = 0 # time index
date_time = []
for day in ds:
    strday = str(day.strftime("%Y%m%d"))            
    # read in the input data    
    fp_stats1 = DATA_DIRECTORY +'Group_C/ec_flux_' + strday + '_0000.dat'
    fp_stats2 = DATA_DIRECTORY +'Group_B/ec_flux_' + strday + '_0000.dat'
    fp_stats3 = DATA_DIRECTORY +'Group_A/ec_flux_' + strday + '_0000.dat'
    if os.path.isfile(fp_stats2) & os.path.isfile(fp_stats3):
    # if os.path.isfile(fp_stats1) & os.path.isfile(fp_stats2) & os.path.isfile(fp_stats2):
        print('start processing:'+ strday)
        # create an empty array for everyday;
        u_ins = np.empty((rpat*24,sonum))*np.nan
        v_ins = np.empty((rpat*24,sonum))*np.nan
        w_ins = np.empty((rpat*24,sonum))*np.nan
        Tsonic_ins = np.empty((rpat*24,sonum))*np.nan
        diag_csat_ins = np.empty((rpat*24,sonum))*np.nan
        C_ins = np.empty((rpat*24,sonum))*np.nan
        q_ins = np.empty((rpat*24,sonum))*np.nan
        sig_irga_ins = np.empty((rpat*24,sonum))*np.nan
        P_ins = np.empty((rpat*24,sonum))*np.nan
        date_num_ins = np.empty((rpat*24,3))*np.nan
        # date_num_ins[:,0]=ds[iday].year
        # date_num_ins[:,1]=ds[iday].month
        # date_num_ins[:,2]=ds[iday].day

        fid1=pd.read_csv(fp_stats1,sep=',',index_col=0,na_values=-9999)
        fid2=pd.read_csv(fp_stats2,sep=',',index_col=0,na_values=-9999)
        fid3=pd.read_csv(fp_stats3,sep=',',index_col=0,na_values=-9999)
    else: 
        continue 
    date_time = fid2.index[round(rpat/2)-1::rpat]
    for j in range(3):
    # for j in [1,2]: # loop of files
        fid = eval('fid'+str(j+1))
        for i in range(4): # loop in order to change the order of column into the same as header
            u_ins[:,i+4*j] = fid[u_header[i+4*j]] # u velocity, m/s
            v_ins[:,i+4*j] = fid[v_header[i+4*j]] # v velocity, m/s
            w_ins[:,i+4*j] = fid[w_header[i+4*j]] # w velocity, m/s
            Tsonic_ins[:,i+4*j] = fid[Tsonic_header[i+4*j]] # sonic temperature, degC
            diag_csat_ins[:,i+4*j] = fid[diag_header[i+4*j]] # diagnostic word for CSAT -- use to set flag
            if co2_header[i+4*j] in list(fid.columns):
                C_ins[:,i+4*j]=fid[co2_header[i+4*j]] # CO2 density, mg/m3
            else:
                C_ins[:,i+4*j]=np.nan
            if h2o_header[i+4*j] in list(fid.columns):
                q_ins[:,i+4*j]=fid[h2o_header[i+4*j]] # water vapor density, g/m3
            else:
                q_ins[:,i+4*j]=np.nan
            if sig_header[i+4*j] in list(fid.columns):
                sig_irga_ins[:,i+4*j]=fid[sig_header[i+4*j]]
            else:
                sig_irga_ins[:,i+4*j]=np.nan
            if press_header[i+4*j] in list(fid.columns):
                P_ins[:,i+4*j]=fid[press_header[i+4*j]] # pressure, kPa
            else:
                P_ins[:,i+4*j]=np.nan 

            ## change unit
            q_ins  = q_ins*1e-3            # kg/m3
            C_ins  = C_ins*1e-6            # kg/m3
            Tsonic_ins = Tsonic_ins + 273.15       # K
            P_ins  = P_ins*1e3             # Pa   

    t_id += 1
    # save results
    if do_saving==1:
        for var in out_var:
        #     globals()[var].to_json(OUT_DIRECTORY + var +'_920.json')
            a_file = open(OUT_DIRECTORY + var + '_' + strday +'.pkl', "wb")
            pickle.dump(eval(var), a_file)
            a_file.close()
            print(var)
        a_file = open(OUT_DIRECTORY + 'date_time_' + strday +'.pkl', "wb")
        pickle.dump(date_time, a_file)
        a_file.close()
    print('finish:'+ strday)
    end = time.time()
    print('elapsed time:'+ str(end - start))
# a_file = open(OUT_DIRECTORY + 'date_time.pkl', "wb")
# pickle.dump(date_time, a_file)
# a_file.close()

In [15]:
pd.DataFrame(u_ins)

,0,1,2,3,4,5,6,7,8,9,10,11
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
863995,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
863996,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
863997,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
863998,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
